# FPGA Emulation Using Intel® oneAPI Base Toolkit (Base Kit)

##### Sections
- [Using Intel® oneAPI Base Toolkit (Base Kit) with Intel FPGAs](#Using-Intel®-oneAPI-Base-Toolkit-(Base-Kit)-with-Intel-FPGAs)
- [Emulation](#Emulation)
- [Device and Host Split Compilation](#Device-Host-Split-Compilation)
- [References](#References)

## Learning Objectives

* Understand the development and compilation flow for Intel® FPGAs with the Intel® oneAPI Toolkits
* Evaluate functional validity of code through quick emulation.
* Learn code splitting techniques for fast-recompilation optimization.

***
# Using Intel® oneAPI Base Toolkit (Base Kit) with Intel FPGAs

The development flow for Intel FPGAs with Intel® oneAPI Base Toolkit involves several stages and serves the following purposes (without having to endure the lengthy compile to a full FPGA executable each time):
* Ensure functional correctness of your code.
* Ensure the custom hardware built to implement your code has optimal performance

The following diagram illustrates the FPGA development flow:

<img src="Assets/fpga_flow.png">

- __Emulation__: Validates code functionality by compiling on the CPU to simulate computation.
- __Optimization Report Generation__: Generates an optimization report that describes the structures generated on the FPGA, identifies performance bottlenecks, and estimates resource utilization. 
- __Bitstream Compilation__: Produces the real FPGA bitstream/image to execute on the target FPGA platform.
- __Runtime Analysis__: Generates output files containing the following metrics and performance data:
    - Total speedup
    - Fraction of code accelerated
    - Number of loops and functions offloaded
    - A call tree showing offloadable and accelerated regions

***
# Emulation

### Hough Transform

The Hough Transform is a feature extraction technique used in computer vision applications. After processing an image with an edge-detection algorithm such as a Sobel filter, you are left with a monochrome (black/white) image. This technique is further useful for many detection algorithms to consider the image as a set of lines. However, an image of black and white pixels is not a convenient or useful representation of these lines to algorithms such as object detection. The Hough Transform technique transforms pixels to a set of “line votes.”


The following code sample implements an in-house Hough Transform algorithm on a given image. 

|             Input              |       Expected Output      |
|--------------------------------|----------------------------|
|  <img src="Assets/pic.bmp">    | <img src="Assets/emulation_output.png">  |

__1) Examine the code and click ▶ to save it to a file.__

In [ ]:
%%writefile src/original/hough_transform.cpp
//==============================================================
// Copyright © 2020 Intel Corporation
//
// SPDX-License-Identifier: MIT
// =============================================================
#include <vector>
#include <CL/sycl.hpp>
#include <CL/sycl/INTEL/fpga_extensions.hpp>
#include <chrono>
#include <fstream>

// This file defines the sin and cos values for each degree up to 180
#include "../../util/sin_cos_values.h"

#define WIDTH 180
#define HEIGHT 120
#define IMAGE_SIZE WIDTH*HEIGHT
#define THETAS 180
#define RHOS 217 //Size of the image diagonally: (sqrt(180^2+120^2))
#define NS (1000000000.0) // number of nanoseconds in a second

using namespace std;
using namespace cl;

// This function reads in a bitmap file and outputs an array of pixels
void read_image(char *image_array);

class Hough_Transform_kernel;

int main() {

  //Declare arrays
  char pixels[IMAGE_SIZE];
  short accumulators[THETAS*RHOS*2];

  //Initialize the accumulators
  std:fill(accumulators, accumulators + THETAS*RHOS*2, 0);

  //Read the bitmap file and get a vector of pixels
  read_image(pixels);

  //Block off this code
  //Putting all SYCL work within here ensures it concludes before this block
  //  goes out of scope. Destruction of the buffers is blocking until the
  //  host retrieves data from the buffer.
  {
    //Profiling setup
    //Set things up for profiling at the host
    chrono::high_resolution_clock::time_point t1_host, t2_host;
    sycl::event queue_event;
    sycl::cl_ulong t1_kernel, t2_kernel;
    double time_kernel;
    auto property_list = sycl::property_list{sycl::property::queue::enable_profiling()};

    //Buffer setup
    //Define the sizes of the buffers
    //The sycl buffer creation expects a type of sycl:: range for the size
    sycl::range<1> num_pixels{IMAGE_SIZE};
    sycl::range<1> num_accumulators{THETAS*RHOS*2};
    sycl::range<1> num_table_values{180};

    //Create the buffers which will pass data between the host and FPGA
    sycl::buffer<char, 1> pixels_buf(pixels, num_pixels);
    sycl::buffer<short, 1> accumulators_buf(accumulators,num_accumulators);
    sycl::buffer<float, 1> sin_table_buf(sinvals,num_table_values);
    sycl::buffer<float, 1> cos_table_buf(cosvals,num_table_values);
  
    //Device selection
    //Explicitly compile for the FPGA_EMULATOR, CPU_HOST, or FPGA
    #if defined(FPGA_EMULATOR)
      sycl::INTEL::fpga_emulator_selector device_selector;
    #elif defined(CPU_HOST)
      sycl::host_selector device_selector;
    #else
      sycl::INTEL::fpga_selector device_selector;
    #endif

    //Create queue
    sycl::queue device_queue(device_selector,NULL,property_list);
  
    //Query platform and device
    sycl::platform platform = device_queue.get_context().get_platform();
    sycl::device device = device_queue.get_device();
    std::cout << "Platform name: " <<  platform.get_info<sycl::info::platform::name>().c_str() << std::endl;
    std::cout << "Device name: " <<  device.get_info<sycl::info::device::name>().c_str() << std::endl;

    //Device queue submit
    queue_event = device_queue.submit([&](sycl::handler &cgh) {
      //Uncomment if you need to output to the screen within your kernel
      //sycl::stream os(1024,128,cgh);
      //Example of how to output to the screen
      //os<<"Hello world "<<8+5<<sycl::endl;
    
      //Create accessors
      auto _pixels = pixels_buf.get_access<sycl::access::mode::read>(cgh);
      auto _sin_table = sin_table_buf.get_access<sycl::access::mode::read>(cgh);
      auto _cos_table = cos_table_buf.get_access<sycl::access::mode::read>(cgh);
      auto _accumulators = accumulators_buf.get_access<sycl::access::mode::read_write>(cgh);

      //Call the kernel
      cgh.single_task<class Hough_transform_kernel>([=]() {
        for (uint y=0; y<HEIGHT; y++) {
          for (uint x=0; x<WIDTH; x++){
            unsigned short int increment = 0;
            if (_pixels[(WIDTH*y)+x] != 0) {
              increment = 1;
            } else {
              increment = 0;
            }
            for (int theta=0; theta<THETAS; theta++){
              int rho = x*_cos_table[theta] + y*_sin_table[theta];
              _accumulators[(THETAS*(rho+RHOS))+theta] += increment;
            }
          }
        }
   
      });
  
    });

    //Wait for the kernel to complete before reporting the profiling
    device_queue.wait();

    // Report kernel execution time and throughput
    t1_kernel = queue_event.get_profiling_info<sycl::info::event_profiling::command_start>();
    t2_kernel = queue_event.get_profiling_info<sycl::info::event_profiling::command_end>();
    time_kernel = (t2_kernel - t1_kernel) / NS;
    std::cout << "Kernel execution time: " << time_kernel << " seconds" << std::endl;
  }

  //Test the results against the golden results
  ifstream myFile;
  myFile.open("util/golden_check_file.txt",ifstream::in);
  ofstream checkFile;
  checkFile.open("util/compare_results.txt",ofstream::out);
  vector<int> myList;

  int number;
  while (myFile >> number) {
    myList.push_back(number);
  }
	
  bool failed = false;
  for (int i=0; i<THETAS*RHOS*2; i++) {
    if ((myList[i]>accumulators[i]+1) || (myList[i]<accumulators[i]-1)) {
      failed = true;
      checkFile << "Failed at " << i << ". Expected: " << myList[i] << ", Actual: "
	      << accumulators[i] << endl;
    }
  }

  myFile.close();
  checkFile.close();

  if (failed) {printf("FAILED\n");}
  else {printf("VERIFICATION PASSED!!\n");}
  return 1;
}

// This function reads a bitmap file and places it into a vector for processing 

//Struct of 3 bytes for R,G,B components
typedef struct __attribute__((__packed__)) {
  unsigned char  b;
  unsigned char  g;
  unsigned char  r;
} PIXEL;

void read_image(char *image_array) {
  //Declare a vector to hold the pixels read from the image
  //The image is 720x480, so the CPU runtimes are not too long for emulation
  PIXEL im[WIDTH*HEIGHT];
	
  //Open the image file for reading
  ifstream img;
  img.open("Assets/pic.bmp",ios::in);

  //The next part reads the image file into memory
  
  //Bitmap files have a 54-byte header. Skip these bits
  img.seekg(54,ios::beg);
    
  //Loop through the img stream and store pixels in an array
  for (uint i = 0; i < WIDTH*HEIGHT; i++) {
    img.read(reinterpret_cast<char*>(&im[i]),sizeof(PIXEL));
	      
    //The image is black and white (passed through a Sobel filter already)
    //Store 1 in the array for a white pixel, 0 for a black pixel
    if (im[i].r==0 && im[i].g==0 && im[i].b==0) {
      image_array[i] = 0;
    } else {
      image_array[i] = 1;
    }
  }
}

__2) Compile the code to target the FPGA emulator.__


<img src="Assets/emulator_command.png">

You can also add a "-o" flag to define the output filename.

__Make the following code section active (you will see a blue bar beside the section), and click ▶.__ This compiles the code into an executable targeting the FPGA emulator, and then runs the emulated code. You will see output from the _std::cout_ statements within the code.

In [ ]:
!  /bin/echo "##" $(whoami) is performing Hough Transform compilation emulation notebook.
! dpcpp -fintelfpga src/original/hough_transform.cpp -DFPGA_EMULATOR -o bin/hough_transform.emu
! bin/hough_transform.emu

__You should have seen output that looked like the statement below:__
***
VERIFICATION PASSED!!
***

## Device-Host Split Compilation

Intel® oneAPI DPC++/C++ Compiler supports ahead-of-time (AoT) compilation for FPGA, which means that an FPGA device image is generated at compile time. The FPGA device image generation process can take hours to complete. If you make a change that is exclusive to the host code, it is more efficient to recompile your host code only by re-using the existing FPGA device image and circumventing the time-consuming device compilation process.

You can achieve this functionality without changing the above code by passing the __-reuse-exe=\<exe_name>__ flag to _dpcpp_ upon compiling. However, this method relies heavily on the compiler to verify that new changes introduced do not affect the device image previously generated. Alternatively, you can use __the device link__ method which involves manually splitting host and kernel(device) code into separate files, compiling, and linking at the end. The following image depicts a high-level explanation of the process. 

[<img src="Assets/device_link_img.png">](https://github.com/oneapi-src/oneAPI-samples/tree/master/DirectProgramming/DPC%2B%2BFPGA/Tutorials/GettingStarted/fast_recompile)

Refer to the [fast_recompile](https://github.com/oneapi-src/oneAPI-samples/tree/master/DirectProgramming/DPC%2B%2BFPGA/Tutorials/GettingStarted/fast_recompile) tutorial for more info on this feature. 

Though the source code below is structurally different, it is functionally equivalent to the original solution above. This can be verified by running the emulation compilation flow again. 

__1) Make the code section of the notebook below active and press ▶ to write the device (kernel) and host code to their respective files.__

#### Kernel/Device Code Header File

In [ ]:
%%writefile src/split/hough_transform_kernel.hpp
//==============================================================
// Copyright © 2020 Intel Corporation
//
// SPDX-License-Identifier: MIT
// =============================================================

#include <vector>
#include <CL/sycl.hpp>
#include <CL/sycl/INTEL/fpga_extensions.hpp>
#include "../../util/sin_cos_values.h"

#define WIDTH 180
#define HEIGHT 120
#define IMAGE_SIZE WIDTH*HEIGHT
#define THETAS 180
#define RHOS 217 //Size of the image diagonally: (sqrt(180^2+120^2))
#define NS (1000000000.0) // number of nanoseconds in a second

using namespace sycl;

void RunKernel(char pixels[], short accumulators[]);

#### Kernel/Device Code Main File

In [ ]:
%%writefile src/split/hough_transform_kernel.cpp
//==============================================================
// Copyright © 2020 Intel Corporation
//
// SPDX-License-Identifier: MIT
// =============================================================

#include "hough_transform_kernel.hpp"

class Hough_transform_kernel;

void RunKernel(char pixels[], short accumulators[])
{
    event queue_event;
    auto my_property_list = property_list{sycl::property::queue::enable_profiling()};

    //Buffer setup: The SYCL buffer creation expects a type of sycl:: range for the size
    range<1> num_pixels{IMAGE_SIZE};
    range<1> num_accumulators{THETAS*RHOS*2};
    range<1> num_table_values{180};

    //Create the buffers which pass data between the host and FPGA
    buffer<char, 1> pixels_buf(pixels, num_pixels);
    buffer<short, 1> accumulators_buf(accumulators,num_accumulators);
    buffer<float, 1> sin_table_buf(sinvals,num_table_values);
    buffer<float, 1> cos_table_buf(cosvals,num_table_values);
  
    // Device selection: Explicitly compile for the FPGA_EMULATOR or FPGA
    #if defined(FPGA_EMULATOR)
        INTEL::fpga_emulator_selector device_selector;
    #else
        INTEL::fpga_selector device_selector;
    #endif

    try {
    
        queue device_queue(device_selector,NULL,my_property_list);
        platform platform = device_queue.get_context().get_platform();
        device my_device = device_queue.get_device();
        std::cout << "Platform name: " <<  platform.get_info<sycl::info::platform::name>().c_str() << std::endl;
        std::cout << "Device name: " <<  my_device.get_info<sycl::info::device::name>().c_str() << std::endl;

        //Submit device queue 
        queue_event = device_queue.submit([&](sycl::handler &cgh) {    
          //Create accessors
          auto _pixels = pixels_buf.get_access<sycl::access::mode::read>(cgh);
          auto _sin_table = sin_table_buf.get_access<sycl::access::mode::read>(cgh);
          auto _cos_table = cos_table_buf.get_access<sycl::access::mode::read>(cgh);
          auto _accumulators = accumulators_buf.get_access<sycl::access::mode::read_write>(cgh);

          //Call the kernel
          cgh.single_task<class Hough_transform_kernel>([=]() {
            for (uint y=0; y<HEIGHT; y++) {
              for (uint x=0; x<WIDTH; x++){
                unsigned short int increment = 0;
                if (_pixels[(WIDTH*y)+x] != 0) {
                  increment = 1;
                } else {
                  increment = 0;
                }
                for (int theta=0; theta<THETAS; theta++){
                  int rho = x*_cos_table[theta] + y*_sin_table[theta];
                  _accumulators[(THETAS*(rho+RHOS))+theta] += increment;
                }
              }
            }
       
          });
      
        });
    } catch (sycl::exception const &e) {
        // Catches exceptions in the host code
        std::cout << "Caught a SYCL host exception:\n" << e.what() << "\n";

        // Most likely the runtime could not find FPGA hardware!
        if (e.get_cl_code() == CL_DEVICE_NOT_FOUND) {
          std::cout << "If you are targeting an FPGA, ensure that your "
                       "system has a correctly configured FPGA board.\n";
          std::cout << "If you are targeting the FPGA emulator, compile with "
                       "-DFPGA_EMULATOR.\n";
        }
        std::terminate();
    }

    // Report kernel execution time and throughput
    cl_ulong t1_kernel = queue_event.get_profiling_info<sycl::info::event_profiling::command_start>();
    cl_ulong t2_kernel = queue_event.get_profiling_info<sycl::info::event_profiling::command_end>();
    double time_kernel = (t2_kernel - t1_kernel) / NS;
    std::cout << "Kernel execution time: " << time_kernel << " seconds" << std::endl;
}

#### Host code Main File

In [ ]:
%%writefile src/split/main.cpp
//==============================================================
// Copyright © 2020 Intel Corporation
//
// SPDX-License-Identifier: MIT
// =============================================================

#include <vector>
#include <CL/sycl.hpp>
#include <chrono>
#include <fstream>
#include "hough_transform_kernel.hpp"

using namespace std;

void read_image(char *image_array); // Funciton for converting a bitmap to an array of pixels
class Hough_transform_kernel;

int main() {
  char pixels[IMAGE_SIZE];
  short accumulators[THETAS*RHOS*2];

  std::fill(accumulators, accumulators + THETAS*RHOS*2, 0);

  read_image(pixels); //Read in the bitmap file and get a vector of pixels

  RunKernel(pixels, accumulators);
 
  ifstream myFile;
  myFile.open("util/golden_check_file.txt",ifstream::in);
  ofstream checkFile;
  checkFile.open("util/compare_results.txt",ofstream::out);
  
  vector<int> myList;
  int number;
  while (myFile >> number) {
    myList.push_back(number);
  }
	
  bool failed = false;
  for (int i=0; i<THETAS*RHOS*2; i++) {
    if ((myList[i]>accumulators[i]+1) || (myList[i]<accumulators[i]-1)) { //Test the results against the golden results
      failed = true;
      checkFile << "Failed at " << i << ". Expected: " << myList[i] << ", Actual: "
	      << accumulators[i] << std::endl;
    }
  }

  myFile.close();
  checkFile.close();

  if (failed) {printf("FAILED\n");}
  else {printf("VERIFICATION PASSED!!\n");}

  return 1;
}

//Struct of 3 bytes for R,G,B components
typedef struct __attribute__((__packed__)) {
  unsigned char  b;
  unsigned char  g;
  unsigned char  r;
} PIXEL;

void read_image(char *image_array) {
  //Declare a vector to hold the pixels read from the image
  //The image is 720x480, so the CPU runtimes are not too long for emulation
  PIXEL im[WIDTH*HEIGHT];
	
  //Open the image file for reading
  ifstream img;
  img.open("Assets/pic.bmp",ios::in);
  
  //Bitmap files have a 54-byte header. Skip these bits
  img.seekg(54,ios::beg);
    
  //Loop through the img stream and store pixels in an array
  for (uint i = 0; i < WIDTH*HEIGHT; i++) {
    img.read(reinterpret_cast<char*>(&im[i]),sizeof(PIXEL));
	      
    //The image is black and white (passed through a Sobel filter already)
    //Store 1 in the array for a white pixel, 0 for a black pixel
    if (im[i].r==0 && im[i].g==0 && im[i].b==0) {
      image_array[i] = 0;
    } else {
      image_array[i] = 1;
    }
  }
}

__2) Compile the code to verify functional equivalency.__
__Make the code section below active and press ▶ to compile the code observe the output.__

In [ ]:
!  /bin/echo "##" $(whoami) is performing Hough Transform split-compilation emulation notebook.
! dpcpp -fintelfpga -DFPGA_EMULATOR src/split/main.cpp src/split/hough_transform_kernel.cpp -o bin/hough_transform_split.emu
! bin/hough_transform_split.emu

__If you see the following final output, it is understood that the two solutions are functionally equivalent:__
***
VERIFICATION PASSED!!
***


## Summary

The FPGA Emulation stage is critical for testing functionality of your design while doing iterative development for FPGAs. The small compilation time allows you to constantly make changes to designs and validate algorithms. 

***
## References

Refer to the following resources for more information about SYCL programming:

#### FPGA-specific Documentation

* [Website hub for using FPGAs with oneAPI](https://software.intel.com/content/www/us/en/develop/tools/oneapi/components/fpga.html)
* [Intel® oneAPI Programming Guide](https://software.intel.com/content/www/us/en/develop/documentation/oneapi-programming-guide/top.html)
* [Intel® oneAPI DPC++ FPGA Optimization Guide](https://software.intel.com/content/www/us/en/develop/documentation/oneapi-fpga-optimization-guide/top.html)
* [oneAPI Fast Recompile Tutorial Documentation](https://github.com/oneapi-src/oneAPI-samples/tree/master/DirectProgramming/DPC%2B%2BFPGA/Tutorials/GettingStarted/fast_recompile)
* [oneAPI FPGA Tutorials GitHub](https://github.com/oneapi-src/oneAPI-samples/tree/master/DirectProgramming/DPC%2B%2BFPGA/Tutorials)

#### Intel® oneAPI Toolkit documentation
* [Intel® oneAPI main page](https://software.intel.com/oneapi "oneAPI main page")
* [Intel® oneAPI programming guide](https://software.intel.com/sites/default/files/oneAPIProgrammingGuide_3.pdf "oneAPI programming guide")
* [Intel® DevCloud Signup](https://software.intel.com/en-us/devcloud/oneapi "Intel DevCloud")
* [Intel® DevCloud Connect](https://devcloud.intel.com/datacenter/connect) 
* [Get Started with the Intel® oneAPI Base Toolkit for Linux*](https://software.intel.com/content/www/us/en/develop/documentation/get-started-with-intel-oneapi-base-linux/top.html)
* [Get Started with the Intel® oneAPI Base Toolkit for Windows*](https://software.intel.com/content/www/us/en/develop/documentation/get-started-with-intel-oneapi-base-windows/top.html)
* [oneAPI Specification elements](https://www.oneapi.com/spec/)

#### SYCL 
* [SYCL* Specification (for version 1.2.1)](https://www.khronos.org/registry/SYCL/specs/sycl-1.2.1.pdf)

#### DPC++
* [Data Parallel C++ Book](https://link.springer.com/book/10.1007%2F978-1-4842-5574-2)

#### Modern C++
* [CPPReference](https://en.cppreference.com/w/)
* [CPlusPlus](http://www.cplusplus.com/)